<a href="https://colab.research.google.com/github/Hoangthang017/CS336.L11/blob/master/Do_an_cuoi_ki/main_search_football.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# clone github
!git clone https://github.com/Hoangthang017/CS336.L11.git
!pip install underthesea

      Successfully uninstalled torch-1.7.0+cu101


In [2]:
from glob import glob
import pandas as pd
import pickle
import re
import string
import numpy as np
from underthesea import word_tokenize
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer

In [4]:
# unzip data
import zipfile
with zipfile.ZipFile("/content/CS336.L11/Do_an_cuoi_ki/dataset_football/bong_da_v1.zip", 'r') as zip_ref:
    zip_ref.extractall("/content/CS336.L11/Do_an_cuoi_ki/dataset_football/")
# load dataset
files_path = glob("/content/CS336.L11/Do_an_cuoi_ki/dataset_football/Bong Da/*/*.txt")

In [5]:
print(files_path)

['/content/CS336.L11/Do_an_cuoi_ki/dataset_football/Bong Da/Việt Nam/7 phát ngôn gây sốc của bầu Đệ trước khi rời Thanh Hoá.txt', '/content/CS336.L11/Do_an_cuoi_ki/dataset_football/Bong Da/Việt Nam/Tường thuật Sài Gòn 0-1 Viettel_ Than.QN 0-4 Hà Nội.txt', '/content/CS336.L11/Do_an_cuoi_ki/dataset_football/Bong Da/Việt Nam/U22 Việt Nam chênh lệch đẳng cấp, ông Park đau đầu.txt', '/content/CS336.L11/Do_an_cuoi_ki/dataset_football/Bong Da/Việt Nam/Đội trưởng ra đi, Sài Gòn FC mất 18 cầu thủ!.txt', '/content/CS336.L11/Do_an_cuoi_ki/dataset_football/Bong Da/Việt Nam/Ông Nguyễn Văn Đệ nghỉ làm ‘bầu’ ở Thanh Hoá.txt', '/content/CS336.L11/Do_an_cuoi_ki/dataset_football/Bong Da/Việt Nam/Những sao số hưởng lương khủng hơn Geovane trong lịch sử V.League.txt', '/content/CS336.L11/Do_an_cuoi_ki/dataset_football/Bong Da/Việt Nam/HLV Nishino doạ Việt Nam, hứa tạo ra ĐT Thái Lan phiên bản mạnh nhất.txt', '/content/CS336.L11/Do_an_cuoi_ki/dataset_football/Bong Da/Việt Nam/HLV Trương Việt Hoàng_ _Chúng 

In [6]:
# unzip data
import zipfile
with zipfile.ZipFile("/content/CS336.L11/Do_an_cuoi_ki/dataset_tfidf/tfidf_vector.zip", 'r') as zip_ref:
    zip_ref.extractall("/content/CS336.L11/Do_an_cuoi_ki/dataset_tfidf/")
# load data from gg drive 
df_tfidf = pd.read_csv('/content/CS336.L11/Do_an_cuoi_ki/dataset_tfidf/tfidf_vector.csv', index_col=0)

In [7]:
print(df_tfidf)

               0    1    2    3    4  ...  1193  1194      1195  1196  1197
00           0.0  0.0  0.0  0.0  0.0  ...   0.0   0.0  0.000000   0.0   0.0
005          0.0  0.0  0.0  0.0  0.0  ...   0.0   0.0  0.000000   0.0   0.0
007          0.0  0.0  0.0  0.0  0.0  ...   0.0   0.0  0.000000   0.0   0.0
01           0.0  0.0  0.0  0.0  0.0  ...   0.0   0.0  0.000000   0.0   0.0
02           0.0  0.0  0.0  0.0  0.0  ...   0.0   0.0  0.000000   0.0   0.0
...          ...  ...  ...  ...  ...  ...   ...   ...       ...   ...   ...
ứng_cử_viên  0.0  0.0  0.0  0.0  0.0  ...   0.0   0.0  0.070974   0.0   0.0
ứng_dụng     0.0  0.0  0.0  0.0  0.0  ...   0.0   0.0  0.000000   0.0   0.0
ứng_phó      0.0  0.0  0.0  0.0  0.0  ...   0.0   0.0  0.000000   0.0   0.0
ứng_viên     0.0  0.0  0.0  0.0  0.0  ...   0.0   0.0  0.000000   0.0   0.0
ứng_xử       0.0  0.0  0.0  0.0  0.0  ...   0.0   0.0  0.000000   0.0   0.0

[15210 rows x 1198 columns]


In [8]:
# load vectorizer
vec = open('/content/CS336.L11/Do_an_cuoi_ki/dataset_tfidf/vectorizer.pk', 'rb')
vectorizer = pickle.load(vec)
# vec.close()

In [ ]:
# import sys
# sys.path.insert(1, '/content/CS336.L11/Do_an_cuoi_ki')

In [9]:
# hàm tiền xử lí văn bản
def text_preprocess(document):
  # loại bỏ html nếu có
  document_test = re.sub(r'@\w+', '', document)

  # viết thường tất cả 
  document_test = document_test.lower()

  # bỏ dấu câu 
  document_test = re.sub(r'[%s]' % re.escape(string.punctuation), ' ', document_test)

  # thay tern v_league
  document_test = word_tokenize(document_test,"text").replace("v league","v_league")
  
  # xóa bỏ các khoảng trắng thừa 
  document_test = re.sub(r'\s{2,}', ' ', document_test)

  return document_test

In [12]:
def get_similar_articles(q, df, files_path, vectorizer):
  print("Câu truy vấn:", q)
  # tiền xử lí câu truy vấn
  q = [text_preprocess(q)]
  # print(q_vec)
  transformer = TfidfTransformer()
  loaded_vec = CountVectorizer(decode_error="replace",vocabulary=pickle.load(open("/content/CS336.L11/Do_an_cuoi_ki/dataset_tfidf/vectorizer.pk", "rb")))
  tfidf = transformer.fit_transform(loaded_vec.fit_transform(q))
  
  sim = {}
  # tính toán độ tương đồng
  for i in range(len(files_path)):
    sim[i] = np.dot(df.loc[:, i].values, q_vec) / np.linalg.norm(df.loc[:, i]) * np.linalg.norm(q_vec)
  # print('độ dài ', len(sim))

  # sắp xếp độ tương đồng
  sim_sorted = sorted(sim.items(), key=lambda x: x[1], reverse=True)
  # print(type(sim_sorted))

  # số lượng bài viết tìm được
  rank = 10
  now = 0

  # in kết quả truy vấn được
  for k, v in sim_sorted:
    print("Độ tương đồng: ", v)
    print("Tiêu đề: ", files_name[k])
    print(files_path[k])
    now += 1
    if (now == rank):
      break

In [11]:
# câu truy vấn
q = 'barcelona'
# gọi hàm truy vấn
get_similar_articles(q,df_tfidf,files_path,vectorizer)

Câu truy vấn: barcelona


KeyError: ignored